In [ ]:
pip install nltk


In [ ]:
import nltk
import numpy as np

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
from nltk import punkt
from nltk.corpus import stopwords
import pandas as pd
ds = pd.read_csv('DataNLP.csv')
ds = ds.applymap(str.lower)
#remove all number
ds['Data'] = ds['Data'].str.replace(r'\d+','')
#remove all punctuation
ds['Data'] = ds['Data'].str.replace('[^\w\s]','')
#Remove WhiteSpaces
ds['Data'] = ds['Data'].str.strip()


We need to remove all the lowering the data helps to make the dataset become consistent. We also need to remove number and punctuation as it isn't relevant. Also we remove whitespaces to remove trailing and ending space.

Tokenization and Stopword removal

In [ ]:
stop_words = set(stopwords.words('english'))
ds['Edit'] = ds['Data'].apply(nltk.word_tokenize)

ds['Edit']= ds['Edit'].apply(lambda x: [i for i in x if i not in stop_words])
ds

,Data,Label,Edit
0,this was a great movie with a good cast all of...,positive,"[great, movie, good, cast, hitting, cylinders]"
1,even if youre a huge sandler fan please dont b...,negative,"[even, youre, huge, sandler, fan, please, dont..."
2,a movie of outstanding brilliance and a poigna...,positive,"[movie, outstanding, brilliance, poignant, unu..."
3,i had the misfortune to watch this rubbish on ...,negative,"[misfortune, watch, rubbish, sky, cinema, max,..."
4,i am at a distinct disadvantage here i have no...,negative,"[distinct, disadvantage, seen, first, two, mov..."
5,this program is a lot of fun and the title son...,positive,"[program, lot, fun, title, song, catchy, cant,..."


We use tokenize to split the text into smaller pieces because the common way to process the text happens at the token level and we remove stopwords as it have no important meaning. All of these changes are saved into a new column of dataset called edit, so we can see the diffrent after the preprocessing of data.

Changing into Base form

In [ ]:

#lemmatization
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
ds['Edit'] = ds['Edit'].apply(lambda x: [lemmatizer.lemmatize(y) for y in x])
ds



,Data,Label,Edit
0,this was a great movie with a good cast all of...,positive,"[great, movie, good, cast, hitting, cylinder]"
1,even if youre a huge sandler fan please dont b...,negative,"[even, youre, huge, sandler, fan, please, dont..."
2,a movie of outstanding brilliance and a poigna...,positive,"[movie, outstanding, brilliance, poignant, unu..."
3,i had the misfortune to watch this rubbish on ...,negative,"[misfortune, watch, rubbish, sky, cinema, max,..."
4,i am at a distinct disadvantage here i have no...,negative,"[distinct, disadvantage, seen, first, two, mov..."
5,this program is a lot of fun and the title son...,positive,"[program, lot, fun, title, song, catchy, cant,..."


Using lemmatization we change the input into their base form of word becaise we need to extract the data from the text and insert it into a database.

Changing the label form positive and negative into 1 and 0 as it is easier to predict a number.

In [ ]:
filter = ds['Label'] == 'positive'
dfpos = ds[filter]
dfneg = ds[~filter]

In [ ]:
dfpos = dfpos['Edit']
dfpos

0        [great, movie, good, cast, hitting, cylinder]
2    [movie, outstanding, brilliance, poignant, unu...
5    [program, lot, fun, title, song, catchy, cant,...
Name: Edit, dtype: object

In [ ]:
dfneg = dfneg['Edit']
dfneg

1    [even, youre, huge, sandler, fan, please, dont...
3    [misfortune, watch, rubbish, sky, cinema, max,...
4    [distinct, disadvantage, seen, first, two, mov...
Name: Edit, dtype: object

In [ ]:
for i in range(len(ds['Label'])) :
  if ds['Label'][i] == 'positive':
    ds['Label'][i]=1
  else :
    ds['Label'][i]=0

In [ ]:
ds

,Data,Label,Edit
0,this was a great movie with a good cast all of...,1,"[great, movie, good, cast, hitting, cylinder]"
1,even if youre a huge sandler fan please dont b...,0,"[even, youre, huge, sandler, fan, please, dont..."
2,a movie of outstanding brilliance and a poigna...,1,"[movie, outstanding, brilliance, poignant, unu..."
3,i had the misfortune to watch this rubbish on ...,0,"[misfortune, watch, rubbish, sky, cinema, max,..."
4,i am at a distinct disadvantage here i have no...,0,"[distinct, disadvantage, seen, first, two, mov..."
5,this program is a lot of fun and the title son...,1,"[program, lot, fun, title, song, catchy, cant,..."


Taking all of the word and put it into one place

In [ ]:
def get_all_words(dataset):
    allword=[]
    for i in dataset:
        for y in i:
            allword.append(y)
    return allword
    
all_pos_words = get_all_words(dfpos)
all_neg_words = get_all_words(dfneg)
allword= get_all_words(ds['Edit'])


In [ ]:
allword

['great',
 'movie',
 'good',
 'cast',
 'hitting',
 'cylinder',
 'even',
 'youre',
 'huge',
 'sandler',
 'fan',
 'please',
 'dont',
 'bother',
 'extremely',
 'disappointing',
 'comedy',
 'movie',
 'outstanding',
 'brilliance',
 'poignant',
 'unusual',
 'love',
 'story',
 'misfortune',
 'watch',
 'rubbish',
 'sky',
 'cinema',
 'max',
 'cold',
 'winter',
 'night',
 'distinct',
 'disadvantage',
 'seen',
 'first',
 'two',
 'movie',
 'series',
 'program',
 'lot',
 'fun',
 'title',
 'song',
 'catchy',
 'cant',
 'get',
 'head']

Preparing the data using word2vec

In [ ]:
from gensim.models import Word2Vec 

# Create CBOW model 
model = Word2Vec([allword], min_count = 1, size = 100, window = 5)
#setup the parameter of the model one by1
print(model.wv.vocab)
print(model.wv.syn0)
print(model.wv.syn0.shape)

filename="word2vecembedding.txt"
model.wv.save_word2vec_format(filename,binary=False)

{'great': <gensim.models.keyedvectors.Vocab object at 0x7f84456ff898>, 'movie': <gensim.models.keyedvectors.Vocab object at 0x7f84456ff048>, 'good': <gensim.models.keyedvectors.Vocab object at 0x7f84456ffb70>, 'cast': <gensim.models.keyedvectors.Vocab object at 0x7f84456ff5f8>, 'hitting': <gensim.models.keyedvectors.Vocab object at 0x7f84456ffd30>, 'cylinder': <gensim.models.keyedvectors.Vocab object at 0x7f84456ffc18>, 'even': <gensim.models.keyedvectors.Vocab object at 0x7f84456ffba8>, 'youre': <gensim.models.keyedvectors.Vocab object at 0x7f84456ffb38>, 'huge': <gensim.models.keyedvectors.Vocab object at 0x7f844d757d30>, 'sandler': <gensim.models.keyedvectors.Vocab object at 0x7f844d757cc0>, 'fan': <gensim.models.keyedvectors.Vocab object at 0x7f844d7576a0>, 'please': <gensim.models.keyedvectors.Vocab object at 0x7f844d7579e8>, 'dont': <gensim.models.keyedvectors.Vocab object at 0x7f844d757780>, 'bother': <gensim.models.keyedvectors.Vocab object at 0x7f844d757550>, 'extremely': <gen

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Word2vec is a two layer neural network that is trained to change word into vector so that the words can be understand by a deep learning model.
  From a large corpus of word it will produce a vector space, with each unique  word in a corpus being assign a vector in the space.
  Using a neural network word2vec take input as one-hot vectors, which is a vector with the same length, filled with zero excepet the index that represent the word we want to repesent, assign "1".

the rows of hidden layer weight matrix, are actually the word vector, it operates as a lookup table. The output, is just a word vector of the input word.

and the output layer is just a softmax activation function.


In [ ]:
model.wv.most_similar('movie')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('program', 0.18575544655323029),
 ('get', 0.14780622720718384),
 ('great', 0.13841168582439423),
 ('head', 0.1335858404636383),
 ('song', 0.1091347187757492),
 ('distinct', 0.10216890275478363),
 ('rubbish', 0.09900261461734772),
 ('two', 0.09880907833576202),
 ('huge', 0.09151367098093033),
 ('cant', 0.08475174009799957)]

After we change the word into vector it become somekind of data of words that's saved into "word2vecembedding.txt"

We need to read the "word2vecembedding.txt" and take the value,word, and the index.

In [1]:
import os
embeddings_index = {}
f= open(os.path.join('','word2vecembedding.txt'),encoding="utf-8")
for line in f:
  values = line.split()
  word = values[0]
  coefs= np.asarray(values[1:])
  embeddings_index[word]=coefs
f.close()

FileNotFoundError: ignored

Using keras we take tokenzier and fit the transformed dataset and using pad_sequences we pad the vector so that all of the input have the same width of vector.

In [ ]:
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

tokenizer_obj = Tokenizer()
tokenizer_obj.fit_on_texts(ds['Edit'])
total_reviews = ds['Edit'].values

sequences = tokenizer_obj.texts_to_sequences(ds['Edit'])

max_length = max([len(i) for i in total_reviews])
print(max_length)


#pad
word_index= tokenizer_obj.word_index
review_pad = pad_sequences(sequences,maxlen=max_length)
sentiment = ds['Label'].values
print(review_pad)
print(sentiment.shape)

#define vocab size
vocab_size = len(tokenizer_obj.word_index)+1
print(vocab_size)


11
[[ 0  0  0  0  0  2  1  3  4  5  6]
 [ 7  8  9 10 11 12 13 14 15 16 17]
 [ 0  0  0  0  1 18 19 20 21 22 23]
 [ 0  0 24 25 26 27 28 29 30 31 32]
 [ 0  0  0  0 33 34 35 36 37  1 38]
 [ 0  0 39 40 41 42 43 44 45 46 47]]
(6,)
48


Form all of the matrix above, we transform the word into vector form the word2vec data by picking the corresponding index. 

In [ ]:
num_words = len(word_index)+1
embedding_matrix = np.zeros((num_words,100))
for word, i in word_index.items():
  if i>num_words:
    continue
  embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector

In [ ]:
print(num_words)

48


In [ ]:
pip install keras_metrics

These are the BPNN model using keras, in keras the model automaticly use BPNN so we only need to specify the model, the input layer consist of how many the length is and for the hidden layer we use 32 hidden layer. And 1 dense layer of the output.

In [ ]:
from keras import Sequential
from keras.layers import Dense,Embedding,LSTM,GRU
from keras.layers.embeddings import Embedding
import keras_metrics

EMBEDDING_DIM = 100

model= Sequential()
model.add(Embedding(vocab_size,EMBEDDING_DIM,input_length=max_length))
model.add(GRU(units=32, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy',keras_metrics.precision(), keras_metrics.recall(),keras_metrics.f1_score()])

model.summary()

tracking <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=0> tp
tracking <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=0> fp
tracking <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=0> tp
tracking <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=0> fn
tracking <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=0> tp
tracking <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=0> fp
tracking <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=0> tp
tracking <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=0> fn
Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 11, 100)           4800      
_________________________________________________________________
gru_4 (GRU)                  (None, 32)                12768     
_________________________________________________________________
dense_4 (Dense)              (

We split the data into 4 training and 2 testing, so that we can see if the model could project / see if the model could see if the model could predict negative and positive.

In [ ]:
VALIDATION_SPLIT = 0.4
indices=np.arange(review_pad.shape[0])

review_pad = review_pad[indices]
sentiment = sentiment[indices]
num_validation_samples = int (VALIDATION_SPLIT*review_pad.shape[0])
x_train_pad = review_pad[:-num_validation_samples]
y_train = sentiment[:-num_validation_samples]
x_test_pad = review_pad[-num_validation_samples:]
y_test = sentiment[-num_validation_samples:]

print(x_train_pad.shape)
print(y_train.shape)
print(x_test_pad.shape)
print(y_test.shape)


(4, 11)
(4,)
(2, 11)
(2,)


These are the training model

In [ ]:
themodel = model.fit(x_train_pad,y_train, epochs= 10, validation_data=(x_test_pad,y_test),verbose=2)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 4 samples, validate on 2 samples
Epoch 1/10
 - 1s - loss: 0.6914 - accuracy: 0.5000 - precision: 0.5000 - recall: 1.0000 - f1_score: 0.6667 - val_loss: 0.7085 - val_accuracy: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_f1_score: 0.0000e+00
Epoch 2/10
 - 0s - loss: 0.6902 - accuracy: 0.2500 - precision: 0.3333 - recall: 0.5000 - f1_score: 0.4000 - val_loss: 0.7091 - val_accuracy: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_f1_score: 0.0000e+00
Epoch 3/10
 - 0s - loss: 0.6823 - accuracy: 0.5000 - precision: 0.5000 - recall: 0.5000 - f1_score: 0.5000 - val_loss: 0.7097 - val_accuracy: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_f1_score: 0.0000e+00
Epoch 4/10
 - 0s - loss: 0.6678 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - f1_score: 1.0000 - val_loss: 0.7103 - val_accuracy: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_f1_score: 0.0000e+00
Epoch 5/10
 - 0s - loss: 0

At the last one, We got accuracy of 1.000(100%) , recall of 1.000(100%) and f1_score of 1.000(100%), This score are good enough, because the dataset only contain of 6 data and from that 6 data there's only 4 data for trainning.

https://medium.com/@ksnugroho/dasar-text-preprocessing-dengan-python-a4fa52608ffe

https://medium.com/@datamonsters/text-preprocessing-in-python-steps-tools-and-examples-bf025f872908

https://www.digitalocean.com/community/tutorials/how-to-perform-sentiment-analysis-in-python-3-using-the-natural-language-toolkit-nltk

https://www.geeksforgeeks.org/python-word-embedding-using-word2vec/

https://www.kaggle.com/pierremegret/gensim-word2vec-tutorial

https://towardsdatascience.com/machine-learning-word-embedding-sentiment-classification-using-keras-b83c28087456

https://israelg99.github.io/2017-03-23-Word2Vec-Explained/

